In [1]:
import sys

sys.path.append('..')

from src.tool.execute_chyper import ExecuteCypherTool
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent

model = ChatOpenAI(
    model="qwen3-8B", 
    base_url="http://127.0.0.1:1234/v1", 
    api_key=""
)

tools = [ExecuteCypherTool()]

agent_executor = create_agent(model, tools)

In [2]:

chat_history = [
    SystemMessage(
    """
        You are an expert Medical Fraud Detection AI Agent powered by a Knowledge Graph.
        Your goal is to validate insurance claims against medical rules.

        You have 1 tools to help you analyze the claim is Fraudlent or not.
        1. execute_cypher: Executes Cypher queries against the graph database

        Rule : 
        1. When the claim data from the graph database already has a Status (FRAUD / NORMAL). It means it has been validated before. You should return the existing Status without re-validation.

        Output Formaat: 
        Claim ID: <claim_id>
        Validation Result: <FRAUD/NORMAL>
        Confidence Score: <0-100%>
        Detail Claim Data: <detailed claim data>
        Explanation: <detailed explanation of the validation>
    """,
    ),
]

In [3]:
import re

def clean_llm_response(content: str) -> str:
    """Clean LLM response by removing thinking tags and unwanted content."""
    
    # 1. Remove <think>...</think> tags and their content
    content = re.sub(r'<think>.*?</think>', '', content, flags=re.DOTALL)
    
    # 2. Remove markdown code blocks
    # Note: It's safer to use replace or regex than hard slicing indices
    if content.startswith("```json"):
        content = content.replace("```json", "", 1)
    if content.startswith("```"):
        content = content.replace("```", "", 1)
    if content.endswith("```"):
        content = content[:-3]
        
    # 3. Remove any other XML-like tags
    content = re.sub(r'<[^>]+>', '', content)
    
    return content.strip()

In [4]:
from langchain_core.callbacks import BaseCallbackHandler

# --- 2. Define the Custom Printer Class ---
class ToolExecutionPrinter(BaseCallbackHandler):
    def on_tool_start(self, serialized, input_str, **kwargs):
        """Run when a tool starts running."""
        tool_name = serialized.get("name")
        print(f"\n[LOG] 🛠️  Agent is entering tool: {tool_name}")
        print(f"[LOG]    Input args: {input_str}\n")

    def on_tool_end(self, output, **kwargs):
        """Run when a tool ends running."""
        print(f"[LOG] ✅ Tool execution finished.\n")

In [5]:
user_input = """
C1001
"""

golden_chyper_for_get_claim_data = """
MATCH (c:Claim {id: <claim_id>})
OPTIONAL MATCH (c)-[:HAS_PATIENT]->(patient:Patient)
OPTIONAL MATCH (c)-[:HAS_PRIMARY_PROCEDURE]->(pp:Procedure)
OPTIONAL MATCH (c)-[:HAS_SECONDARY_PROCEDURE]->(sp:Procedure)
OPTIONAL MATCH (c)-[:HAS_CLINICAL_NOTE]->(note:ClinicalNote)
OPTIONAL MATCH (c)-[:SUBMITTED_AT]->(hospital:Hospital)
OPTIONAL MATCH (c)-[:SUBMITTED_BY]->(doctor:Doctor)
OPTIONAL MATCH (c)-[:CODED_AS]->(diagnosis:Diagnosis)
RETURN c.id, c.total_cost, c.status,
       patient.name as patient_name,
       hospital.name as hospital_name,
       doctor.name as doctor_name,
       diagnosis.name as diagnosis_name,
       collect(DISTINCT pp.name) as primary_procedures,
       collect(DISTINCT sp.name) as secondary_procedures,
       note.primary_diagnosis_text,
       note.secondary_diagnosis_text
"""

golden_chyper_to_get_price_procedure_diagnose_based_on_claim_id = """
MATCH (c:Claim {id: <claim_id>})
OPTIONAL MATCH (c)-[:HAS_PRIMARY_PROCEDURE]->(pp:Procedure)
OPTIONAL MATCH (c)-[:HAS_SECONDARY_PROCEDURE]->(sp:Procedure)
OPTIONAL MATCH (c)-[:CODED_AS]->(diagnosis:Diagnosis)
WITH c, diagnosis, 
     collect(DISTINCT pp) as primary_procs,
     collect(DISTINCT sp) as secondary_procs
RETURN 
    c.id,
    c.total_cost,
    diagnosis.name,
    diagnosis.avg_cost as diagnosis_cost,
    reduce(total = 0, proc IN primary_procs | total + COALESCE(proc.avg_cost, 0)) as primary_procs_total,
    reduce(total = 0, proc IN secondary_procs | total + COALESCE(proc.avg_cost, 0)) as secondary_procs_total
"""

golden_query_to_get_diagnose_and_procedure_relation = """
MATCH (d:Diagnosis {code: <diagnosis_id>})-[r]->(p:Procedure)
WITH d, count(r) AS procedure_count,
     collect({procedure: p.name, relationship: type(r), cost: p.avg_cost}) AS procedures
RETURN d.code AS ICD10_Code,
       d.name AS Diagnosis_Name,
       d.severity AS Severity,
       d.avg_cost AS Diagnosis_Cost,
       procedure_count AS Number_of_Procedures,
       procedures AS Associated_Procedures;
"""

golden_query_get_specialisties_doctor = """
MATCH (d:Doctor {id: <doctor_id>})
RETURN d.id AS doctor_id,
       d.name AS doctor_name,
       d.specialization AS specialization;
"""

golden_query_get_specialties_and_facilities_hospital = """
MATCH (h:Hospital {id: <hospital_id>})
OPTIONAL MATCH (h)-[:HAS_SPECIALTY]->(s:Specialty)
OPTIONAL MATCH (h)-[:HAS_FACILITY]->(f:Facility)
RETURN h.id AS hospital_id,
       h.name AS hospital_name,
       h.class AS hospital_class,
       h.location AS location,
       collect(DISTINCT s.name) AS specialties,
       collect(DISTINCT f.name) AS facilities;
"""

message = [
    HumanMessage(f"This is Claim ID: {user_input}"),
    HumanMessage(f"""
    Please follow these steps to analyze the claim. Be objective and allow for reasonable operational variances.

    1. **Data Retrieval**: 
       Execute ${golden_chyper_for_get_claim_data} to get all relevant data. 
       - If the status is already "NORMAL" or "FRAUD", return it directly with 100% confidence. (No further validation needed.)
       - If status is null/NaN, proceed to validation steps below.

    2. **Validation Logic (Apply these rules strictly in order)**:

       a. **Procedure Consistency**: 
          Check if the procedure is clinically appropriate for the diagnosis.
          - *Logic*: Use the relation from {golden_query_to_get_diagnose_and_procedure_relation}.
          - *Guidance*: If the procedure is a standard diagnostic tool for the diagnosis (e.g., MRI for Stroke), it is a MATCH.

       b. **Cost Analysis (The 20% Rule)**: 
          Compare the Claim's Total Cost vs. Ground Truth (Sum of Diagnosis Avg Cost + Procedure Avg Cost).
          - *Logic*: Calculate the deviation: `(Claim_Cost - Ground_Truth) / Ground_Truth`.
          - *Guidance*: 
             - If deviation is **< 20%**: Consider this **NORMAL** operational variance (e.g., room upgrades, extra meds). Do NOT flag as fraud based on cost alone.
             - If deviation is **> 20%**: Flag as **FRAUD** (Cost significantly inflated).
          - Ground Truth Query: {golden_chyper_to_get_price_procedure_diagnose_based_on_claim_id} (or calculate manually using averages).

       c. **Doctor Qualification (GP Exception)**: 
          Check if the doctor is qualified.
          - *Logic*: Use {golden_query_get_specialisties_doctor}.
          - *Guidance*: 
             - **GPs (General Practitioners)** are VALID for initial diagnoses, consultations, and ordering standard scans (like MRI/CT), even for complex conditions like Stroke. 
             - Flag as **FRAUD** only if there is a **hard contradiction** (e.g., a Pediatrician performing Major Surgery, or an Ophthalmologist treating Heart Attack).

       d. **Hospital Capability**: 
          Check if the hospital has relevant facilities.
          - *Logic*: Use {golden_query_get_specialties_and_facilities_hospital}.
          - *Guidance*: Look for broad keyword matches. For example, if Diagnosis is "Stroke", facilities like "ICU", "Neurology", or "Internal Medicine" are sufficient evidence of capability.

    3. **Final Verdict**:
       Based on the above, determine FRAUD or NORMAL.
       - Provide a confidence score (0-100%).
       - Provide the Detailed Claim Data.
       - **Explanation**: You MUST explicitly state the cost deviation percentage in your explanation (e.g., "Cost is 4.5% higher, which is within the acceptable 20% variance").
    """)
]

final_messages = chat_history + message

response = agent_executor.invoke({
    "messages": final_messages,
},
config={
        "callbacks": [ToolExecutionPrinter()]
    }
)

raw_content = response['messages'][-1].content

# 3. Apply the Cleaning Function
final_output = clean_llm_response(raw_content)

# 4. Print the Cleaned Response
print(f"Response: {final_output}")


[LOG] 🛠️  Agent is entering tool: execute_cypher
[LOG]    Input args: {'cypher_query': 'MATCH (c:Claim {id: "C1001"}) OPTIONAL MATCH (c)-[:HAS_PATIENT]->(patient:Patient) OPTIONAL MATCH (c)-[:HAS_PRIMARY_PROCEDURE]->(pp:Procedure) OPTIONAL MATCH (c)-[:HAS_SECONDARY_PROCEDURE]->(sp:Procedure) OPTIONAL MATCH (c)-[:HAS_CLINICAL_NOTE]->(note:ClinicalNote) OPTIONAL MATCH (c)-[:SUBMITTED_AT]->(hospital:Hospital) OPTIONAL MATCH (c)-[:SUBMITTED_BY]->(doctor:Doctor) OPTIONAL MATCH (c)-[:CODED_AS]->(diagnosis:Diagnosis) RETURN c.id, c.total_cost, c.status, patient.name as patient_name, hospital.name as hospital_name, doctor.name as doctor_name, diagnosis.name as diagnosis_name, collect(DISTINCT pp.name) as primary_procedures, collect(DISTINCT sp.name) as secondary_procedures, note.primary_diagnosis_text, note.secondary_diagnosis_text'}

[EXECUTE_CYPHER] Executing query: MATCH (c:Claim {id: "C1001"}) OPTIONAL MATCH (c)-[:HAS_PATIENT]->(patient:Patient) OPTIONAL MATCH (c)-[:HAS_PRIMARY_PROCEDURE]

KeyboardInterrupt: 